In [1]:
print("hello")

hello


In [82]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import CTransformers


In [ ]:
PINECONE_API_KEY = ""
PINECONE_API_ENV = "medical-chatbot"

In [ ]:
!pip install -qU langchain-pinecone

In [59]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                         glob="*.pdf",
                         loader_cls=PyPDFLoader)
    
    documnets = loader.load()

    return documnets

In [60]:
extracted_data = load_pdf("../data")

In [61]:
extracted_data

[Document(metadata={'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf', 'page': 0}, page_content=''),
 Document(metadata={'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf', 'page': 1}, page_content=''),
 Document(metadata={'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf', 'page': 2}, page_content='\t\n\t\n   \nAnatomy and Physiology 2e      SENIOR CONTRIBUTING AUTHORS J. GORDON BETTS, TYLER JUNIOR COLLEGE PETER DESAIX, UNIVERSITY OF NORTH CAROLINA AT CHAPEL HILL EDDIE JOHNSON, CENTRAL OREGON COMMUNITY COLLEGE JODY E. JOHNSON, ARAPAHOE COMMUNITY COLLEGE OKSANA KOROL, AIMS COMMUNITY COLLEGE DEAN KRUSE, PORTLAND COMMUNITY COLLEGE BRANDON POE, SPRINGFIELD TECHNICAL COMMUNITY COLLEGE JAMES A. WISE, HAMPTON UNIVERSITY MARK WOMBLE, YOUNGSTOWN STATE UNIVERSITY KELLY A. YOUNG, CALIFORNIA STATE UNIVERSITY, LONG BEACH   \n'),
 Document(metadata={'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf'

In [ ]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [65]:
text_chunks = text_split(extracted_data)
print("length of my chunk: ",len(text_chunks))

length of my chunk:  8727


In [66]:
def download_higging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return embeddings

In [67]:
embeddings = download_higging_face_embeddings()

C:\Users\HP\AppData\Local\Temp\ipykernel_12804\262939136.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\HP\anaconda3\envs\gbot\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\HP\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment varia

In [68]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [69]:
query_result = embeddings.embed_query("Hello World")
print("Length" ,len(query_result))

Length 384


In [70]:
query_result

[-0.03447723761200905,
 0.031023213639855385,
 0.006734990980476141,
 0.02610895223915577,
 -0.03936200216412544,
 -0.16030248999595642,
 0.06692393124103546,
 -0.006441502831876278,
 -0.04745049029588699,
 0.014758865348994732,
 0.07087529450654984,
 0.05552753433585167,
 0.019193345680832863,
 -0.026251327246427536,
 -0.010109513066709042,
 -0.026940496638417244,
 0.022307435050606728,
 -0.022226642817258835,
 -0.1496925801038742,
 -0.01749304123222828,
 0.007676258217543364,
 0.05435232073068619,
 0.0032544711139053106,
 0.031725890934467316,
 -0.0846213549375534,
 -0.029405983164906502,
 0.05159558728337288,
 0.04812406003475189,
 -0.0033148040529340506,
 -0.05827920883893967,
 0.04196924716234207,
 0.022210638970136642,
 0.1281888484954834,
 -0.022338991984725,
 -0.011656233109533787,
 0.06292837113142014,
 -0.03287629410624504,
 -0.09122603386640549,
 -0.031175388023257256,
 0.052699580788612366,
 0.0470348484814167,
 -0.08420310914516449,
 -0.030056176707148552,
 -0.020744847133

In [ ]:
from langchain_community.vectorstores import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
    api_key=PINECONE_API_KEY,
)
my_index = pc.Index('medical-chatbot')



In [88]:
texts = [doc.page_content for doc in text_chunks]
metadatas = [doc.metadata for doc in text_chunks]

In [100]:
metadatas

[{'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf',
  'page': 2},
 {'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf',
  'page': 2},
 {'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf',
  'page': 3},
 {'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf',
  'page': 3},
 {'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf',
  'page': 3},
 {'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf',
  'page': 3},
 {'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf',
  'page': 3},
 {'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf',
  'page': 4},
 {'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf',
  'page': 4},
 {'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf',
  'page': 4},
 {'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice Universi

In [89]:
embeddings_list = embeddings.embed_documents(texts)

In [93]:
!pip install langchain-pinecone

ERROR: Could not find a version that satisfies the requirement langchin-pinecone (from versions: none)
ERROR: No matching distribution found for langchin-pinecone


In [99]:
from langchain_pinecone import PineconeVectorStore
docsearch = PineconeVectorStore(
    index=my_index,
    embedding=embeddings
)

In [106]:
from langchain_core.documents import Document
for chunk in text_chunks:
    doc = Document(
        page_content=chunk.page_content,
        metadata=chunk.metadata
    )

    docsearch.add_documents([doc])

In [107]:
query="what is axial skeleton?"

In [108]:
results = docsearch.similarity_search(
    query=query,
    k=2
)
print(results)

[Document(id='cd00062e-c704-49a5-8ca9-5cab6d56f705', metadata={'page': 244.0, 'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf'}, page_content='FIGURE 7.2 Axial and Appendicular Skeleton The axial skeleton supports the head, neck, back, and chest and thus forms the vertical\naxis of the body. It consists of the skull, vertebral column (including the sacrum and coccyx), and the thoracic cage, formed by the ribs and\nsternum. The appendicular skeleton is made up of all bones of the upper and lower limbs.\nThe Appendicular Skeleton'), Document(id='c8e59beb-edae-479c-8ad2-7f7ce1e8a339', metadata={'page': 242.0, 'source': '..\\data\\Anatomy and Physiology, OpenStax, Rice University.pdf'}, page_content='INTRODUCTION\nCHAPTER 7\nAxial Skeleton\nCHAPTER OBJECTIVES\nAfter studying this chapter, you will be able to:\n• Describe the functions of the skeletal system and define its two major subdivisions\n• Identify the bones and bony structures of the skull, the cranial s

In [110]:
prompt_template = '''
Use the following piece of information to answer the user's question.
If you don't know the answer, just say you don't know. Don't try to make an answer on yourself.
Context: {context}
question: {question}

Only return the helpful answer below and noting else.
Helpful answer:
'''

In [111]:
prompt = PromptTemplate(
    input_variables=["context","question"],
    template=prompt_template
)
chain_type_kwargs = {"prompt":prompt}

In [116]:
os.path.exists(output_path)

False

In [134]:
llm=CTransformers(model="../model/llama-2-7b-chat.ggmlv3.q2_K.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [135]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=docsearch.as_retriever(search_kwargs={'k':2}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

In [ ]:
while True:
    user_input= input("Input Prompt: ")
    result = qa({"query":user_input})
    print("Response: ",result['result'])

Response:  The axial skeleton refers to the main bones that support the body's vertical axis, including the skull, vertebral column, ribcage, and sternum.
